# Latent Space

> Scripts to visualize and explore the Latent Space

In [36]:
#| default_exp latent_space

In [37]:
#| export
#| hide
import numpy as np
from itertools import combinations
from scipy.spatial import distance
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm
from sklearn.preprocessing import LabelEncoder
from typing import Optional, List, Any, Union

In [38]:
#| hide
from fastcore.test import test_eq, test_close

## Latent Space

In [ ]:
#| export
def plot_latent_space_2d(latent_representations: np.ndarray,  # Precomputed latent representations (numpy array).
                         labels: np.ndarray,                  # Labels for the data points, used for coloring in the plot.
                         figsize: tuple = (12, 9),            # Size of the figure for the plot.
                         save_path: Optional[str] = None,     # Optional path to save the plot image.
                         many_classes: bool = False,          # Flag to use enhanced plotting for many classes.
                         **kwargs: Any                        # Additional keyword arguments for the plotting.
                        ) -> None:
    """
    Plots and optionally saves the latent space representations assuming they are already in 2D.
    """
    # Encode string labels to integers
    label_encoder = LabelEncoder()
    encoded_labels = label_encoder.fit_transform(labels)
    class_names = label_encoder.classes_

    # Use a colormap for better color differentiation
    cmap = cm.get_cmap('tab20', len(class_names))  # You can change 'tab20' to any other colormap
    markers = ['o', 's', '^', 'v', 'D', '<', '>', 'p', '*', 'h', 'H', '8']  # Marker styles
    fig, ax = plt.subplots(figsize=figsize)

    if many_classes:
        for class_idx, class_name in enumerate(class_names):
            class_mask = (encoded_labels == class_idx)
            color = cmap(class_idx)
            marker = markers[class_idx % len(markers)]
            ax.scatter(latent_representations[class_mask, 0], latent_representations[class_mask, 1],
                       label=class_name, marker=marker, color=color, s=30, **kwargs)
        ax.legend(title="Classes")
    else:
        unique_labels = np.unique(encoded_labels)
        for class_idx, class_name in zip(unique_labels, class_names):
            class_mask = (encoded_labels == class_idx)
            color = cmap(class_idx)
            ax.scatter(latent_representations[class_mask, 0], latent_representations[class_mask, 1],
                       label=class_name, color=color, s=30, **kwargs)
        ax.legend(title="Classes")

    ax.set_title('2D Latent Space Visualization')
    ax.set_xlabel('Dimension 1')
    ax.set_ylabel('Dimension 2')

    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, bbox_inches='tight')
        print(f"Saved plot to {save_path}")
    plt.show()

In [ ]:
#| export
def plot_combined_latent_space_2d(
        real_data: np.ndarray,                    # Real data samples.
        synthetic_data: np.ndarray,               # Synthetic data samples generated by a model.
        encoder,                                  # Encoder function or model that predicts latent space representations.
        synthetic_labels: Optional[Union[int, List[int]]] = 1,  # Labels for synthetic data. Can be a single label or a list of labels.
        figsize: tuple = (12, 9),                 # Size of the figure.
        save_path: Optional[str] = None,          # Optional path to save the plot image.
        many_classes: bool = False,               # Flag to use enhanced plotting for many classes.
        show_legend: bool = True,                 # Flag to show or hide the legend.
        annotation_mode: str = 'legend'           # Mode for annotation: 'legend' for colored dots, 'numbers' for numeric annotations.
    ) -> None:
    """
    Plots the combined latent space of real and synthetic data.
    Assumes the latent space is already 2D.
    """
    # Concatenate real and synthetic data
    combined_data = np.concatenate([real_data, synthetic_data], axis=0)

    # Create labels for real and synthetic data
    real_labels = np.zeros(real_data.shape[0], dtype=int)
    if isinstance(synthetic_labels, int):
        synthetic_labels = np.full(synthetic_data.shape[0], synthetic_labels, dtype=int)
    synthetic_labels = np.array(synthetic_labels)
    combined_labels = np.concatenate([real_labels, synthetic_labels], axis=0)

    # Generate latent representations for the combined dataset
    latent_representations = encoder.predict(combined_data)

    # Handle case where encoder returns a tuple or list
    if isinstance(latent_representations, (list, tuple)):
        latent_representations = latent_representations[0]  # Assuming the first element is the required representation

    # Convert latent_representations to a NumPy array if it is a list
    latent_representations = np.array(latent_representations)

    # Ensure the latent space is 2D
    assert latent_representations.shape[1] == 2, "The latent space representations must be 2D."

    # Create a color map with enough colors
    cmap = plt.get_cmap('tab20', len(np.unique(combined_labels)))
    norm = mcolors.BoundaryNorm(boundaries=np.arange(len(np.unique(combined_labels))+1)-0.5, ncolors=len(np.unique(combined_labels)))

    # Plotting
    plt.figure(figsize=figsize)
    scatter = plt.scatter(latent_representations[:, 0], latent_representations[:, 1], c=combined_labels, cmap=cmap, norm=norm, alpha=0.7)
    
    if annotation_mode == 'legend' and show_legend:
        # Create custom legend
        unique_labels = np.unique(combined_labels)
        custom_handles = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=cmap(norm(label)), markersize=10) for label in unique_labels]
        custom_labels = ["Real"] + [f"Synthetic {label}" for label in np.unique(synthetic_labels)]
        plt.legend(custom_handles, custom_labels, title="Classes", loc='upper right')  # Place legend in the upper right corner
    
    elif annotation_mode == 'numbers':
        # Annotate only synthetic points with their class labels
        for i in range(real_data.shape[0], combined_data.shape[0]):
            plt.annotate(combined_labels[i], (latent_representations[i, 0], latent_representations[i, 1]), fontsize=9, alpha=0.7)

    # Save the plot if a save path is provided
    if save_path:
        plt.savefig(save_path)
    else:
        plt.show()

In [ ]:
#| export
def plot_latent_space_with_feature_distributions(latent_representations: np.ndarray,
                                                 labels: np.ndarray,
                                                 features: Optional[np.ndarray] = None,
                                                 feature_names: Optional[list] = None,
                                                 figsize: tuple = (12, 12),  # Adjusted to be a square
                                                 save_path: Optional[str] = None,
                                                 many_classes: bool = False,
                                                 show_legend: bool = True,
                                                 legend_fontsize: int = 8,
                                                 **kwargs: Any) -> None:
    """
    Plots the latent space with class colors and normalized vertical and horizontal feature distributions in separate subplots.
    """
    # Encode string labels to integers
    label_encoder = LabelEncoder()
    encoded_labels = label_encoder.fit_transform(labels)
    class_names = label_encoder.classes_

    # Use a colormap for better color differentiation
    cmap = plt.get_cmap('tab20')  # You can change 'tab20' to any other colormap
    markers = ['o', 's', '^', 'v', 'D', '<', '>', 'p', '*', 'h', 'H', '8']  # Marker styles

    # Create subplots
    fig = plt.figure(figsize=figsize)
    # Adjust height_ratios and width_ratios to ensure square latent space plot
    gs = fig.add_gridspec(2, 2, height_ratios=[5, 1], width_ratios=[5, 1])

    ax1 = fig.add_subplot(gs[0, 0])  # Latent space plot
    ax2 = fig.add_subplot(gs[1, 0])  # Horizontal distribution plot
    ax3 = fig.add_subplot(gs[0, 1], sharey=ax1)  # Vertical distribution plot

    # Plot the latent representations with class colors
    if many_classes:
        for class_idx, class_name in enumerate(class_names):
            class_mask = (encoded_labels == class_idx)
            color = cmap(class_idx / len(class_names))
            marker = markers[class_idx % len(markers)]
            ax1.scatter(latent_representations[class_mask, 0], latent_representations[class_mask, 1],
                        label=class_name, marker=marker, color=color, s=30, **kwargs)
    else:
        unique_labels = np.unique(encoded_labels)
        for class_idx, class_name in zip(unique_labels, class_names):
            class_mask = (encoded_labels == class_idx)
            color = cmap(class_idx / len(class_names))
            ax1.scatter(latent_representations[class_mask, 0], latent_representations[class_mask, 1],
                        label=class_name, color=color, s=30, **kwargs)

    ax1.set_title('2D Latent Space Visualization')
    ax1.set_xlabel('Dimension 1')
    ax1.set_ylabel('Dimension 2')
    ax1.set_aspect('equal', 'box')  # Ensure the latent space plot is square
    if show_legend:
        ax1.legend(title="Classes", fontsize=legend_fontsize)

    if features is not None and feature_names is not None:
        # Horizontal distribution
        x_min, x_max = np.min(latent_representations[:, 0]), np.max(latent_representations[:, 0])
        x_bins = np.linspace(x_min, x_max, 100)

        for feature_index, feature_name in enumerate(feature_names):
            feature_values = features[:, feature_index]
            avg_feature_values = []

            for i in range(len(x_bins) - 1):
                mask = (latent_representations[:, 0] >= x_bins[i]) & (latent_representations[:, 0] < x_bins[i+1])
                avg_feature_values.append(np.mean(feature_values[mask]) if np.sum(mask) > 0 else 0)

            avg_feature_values = np.array(avg_feature_values)
            # Normalize the feature values
            avg_feature_values = (avg_feature_values - np.min(avg_feature_values)) / (np.max(avg_feature_values) - np.min(avg_feature_values) + 1e-8)
            # Plot the normalized distributions
            ax2.plot((x_bins[:-1] + x_bins[1:]) / 2, avg_feature_values, label=f'{feature_name} distribution')

        ax2.set_title('Normalized Horizontal Feature Distributions')
        ax2.set_xlabel('Dimension 1')
        ax2.set_ylabel('Normalized Feature Value')
        if show_legend:
            ax2.legend(fontsize=legend_fontsize)

        # Vertical distribution
        y_min, y_max = np.min(latent_representations[:, 1]), np.max(latent_representations[:, 1])
        y_bins = np.linspace(y_min, y_max, 100)

        for feature_index, feature_name in enumerate(feature_names):
            feature_values = features[:, feature_index]
            avg_feature_values = []

            for i in range(len(y_bins) - 1):
                mask = (latent_representations[:, 1] >= y_bins[i]) & (latent_representations[:, 1] < y_bins[i+1])
                avg_feature_values.append(np.mean(feature_values[mask]) if np.sum(mask) > 0 else 0)

            avg_feature_values = np.array(avg_feature_values)
            # Normalize the feature values
            avg_feature_values = (avg_feature_values - np.min(avg_feature_values)) / (np.max(avg_feature_values) - np.min(avg_feature_values) + 1e-8)
            # Plot the normalized distributions
            ax3.plot(avg_feature_values, (y_bins[:-1] + y_bins[1:]) / 2, label=f'{feature_name} distribution')

        ax3.set_title('Normalized Vertical Feature Distributions')
        ax3.set_xlabel('Normalized Feature Value')
        ax3.set_ylabel('Dimension 2')
        if show_legend:
            ax3.legend(fontsize=legend_fontsize)

    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, bbox_inches='tight')
        print(f"Saved plot to {save_path}")
    plt.show()

In [ ]:
#| export
def reduce_dimensions_plot_latent_space(latent_representations: np.ndarray,  # Precomputed latent representations (numpy array).
                      labels: np.ndarray,                  # Labels for the data points, used for coloring in the plot.
                      techniques: List[str] = ['PCA'],     # Techniques to use for reduction ('PCA', 't-SNE', 'UMAP', 'LDA').
                      n_components: int = 2,               # Number of dimensions to reduce to (1, 2, or 3).
                      figsize: tuple = (12, 9),            # Size of the figure for each subplot.
                      save_path: Optional[str] = None,     # Optional path to save the plot image.
                      many_classes: bool = False,          # Flag to use enhanced plotting for many classes.
                      grid_view: bool = True,              # Flag to plot all techniques in a single grid view.
                      class_names: Optional[List[str]] = None,  # Optional class names for the legend
                      show_legend: bool = True,            # Flag to show or hide the legend
                      **kwargs: Any                        # Additional keyword arguments for dimensionality reduction methods.
                     ) -> None:
    """
    Plots and optionally saves the latent space representations using specified dimensionality reduction techniques.
    Each technique's plot is handled in a separate figure, supporting 1D, 2D, or 3D visualizations.
    """
    # Encode string labels to integers
    label_encoder = LabelEncoder()
    encoded_labels = label_encoder.fit_transform(labels)
    
    # Use provided class names if available
    if class_names:
        class_names = class_names
    else:
        class_names = label_encoder.classes_

    models = {
        'PCA': PCA(n_components=n_components),
        't-SNE': TSNE(n_components=n_components, **kwargs),
        'UMAP': umap.UMAP(n_components=n_components, **kwargs),
        'LDA': LinearDiscriminantAnalysis(n_components=n_components)
    }

    markers = ['o', 's', '^', 'v', 'D', '<', '>', 'p', '*', 'h', 'H', '8']  # Marker styles

    n_techniques = len(techniques)
    if grid_view:
        fig, axs = plt.subplots(1, n_techniques, figsize=(figsize[0] * n_techniques, figsize[1]))
        if n_techniques == 1:
            axs = [axs]
    else:
        axs = [None] * n_techniques

    for idx, technique in enumerate(techniques):
        model = models.get(technique)
        if not model:
            continue  # Skip if model not found in dictionary

        if technique == 'LDA':
            results = model.fit_transform(latent_representations, encoded_labels)
        else:
            results = model.fit_transform(latent_representations)

        if n_components == 3:
            if grid_view:
                ax = fig.add_subplot(1, n_techniques, idx+1, projection='3d')
            else:
                fig = plt.figure(figsize=figsize)
                ax = fig.add_subplot(111, projection='3d')
        else:
            ax = axs[idx] if grid_view else plt.subplots(figsize=figsize)[1]

        if many_classes:
            for class_idx, class_name in enumerate(class_names):
                class_mask = (encoded_labels == class_idx)
                marker = markers[class_idx % len(markers)]
                if n_components == 1:
                    ax.scatter(results[class_mask], np.zeros_like(results[class_mask]), label=class_name, marker=marker, s=30)
                elif n_components == 2:
                    ax.scatter(results[class_mask, 0], results[class_mask, 1], label=class_name, marker=marker, s=30)
                elif n_components == 3:
                    ax.scatter(results[class_mask, 0], results[class_mask, 1], results[class_mask, 2], label=class_name, marker=marker, s=30)
            if show_legend:
                ax.legend(title="Classes")
        else:
            unique_labels = np.unique(encoded_labels)
            for class_idx, class_name in zip(unique_labels, class_names):
                class_mask = (encoded_labels == class_idx)
                if n_components == 1:
                    ax.scatter(results[class_mask], np.zeros_like(results[class_mask]), label=class_name, s=30)
                elif n_components == 2:
                    ax.scatter(results[class_mask, 0], results[class_mask, 1], label=class_name, s=30)
                elif n_components == 3:
                    ax.scatter(results[class_mask, 0], results[class_mask, 1], results[class_mask, 2], label=class_name, s=30)
            if show_legend:
                ax.legend(title="Classes")

        ax.set_title(f'Visualization with {technique}')
        if n_components == 1:
            ax.set_xlabel(f'{technique} Component 1')
        elif n_components == 2:
            ax.set_xlabel(f'{technique} Dimension 1')
            ax.set_ylabel(f'{technique} Dimension 2')
        elif n_components == 3:
            ax.set_xlabel(f'{technique} Dimension 1')
            ax.set_ylabel(f'{technique} Dimension 2')
            ax.set_zlabel(f'{technique} Dimension 3')

        if not grid_view:
            if save_path:
                individual_save_path = f"{save_path}_{technique}.png"
                plt.savefig(individual_save_path, bbox_inches='tight')
                print(f"Saved plot to {individual_save_path}")
            plt.show()

    if grid_view:
        plt.tight_layout()
        if save_path:
            grid_save_path = f"{save_path}_grid.png"
            plt.savefig(grid_save_path, bbox_inches='tight')
            print(f"Saved grid plot to {grid_save_path}")
        plt.show()


In [ ]:
#| test reduce_dimensions_plot_latent_space

# Reshape data to 2D (num_orbits, 6 * num_time_points)
orbit_data_reshaped = orbit_data.reshape(200, -1)

# Use PCA to reduce to a lower-dimensional space (e.g., 10 dimensions)
pca = PCA(n_components=10)
latent_representations = pca.fit_transform(orbit_data_reshaped)

labels = np.random.randint(0, 5, size=200)  # 5 different classes

reduce_dimensions_plot_latent_space(latent_representations, labels, techniques=['UMAP','LDA'])

In [ ]:
#| test reduce_dimensions_plot_latent_space
reduce_dimensions_plot_latent_space(latent_representations, labels, techniques=['PCA'], n_components=1, many_classes=True)

In [ ]:
#| test reduce_dimensions_plot_latent_space
reduce_dimensions_plot_latent_space(latent_representations, labels, techniques=['t-SNE'], n_components=3)

In [ ]:
# | export
def reduce_dimensions_plot_combined_latent_space(
        real_data: np.ndarray,                # Real data samples.
        synthetic_data: np.ndarray,           # Synthetic data samples generated by a model.
        encoder,                              # Encoder function or model that predicts latent space representations.
        techniques: List[str] = ['PCA'],      # Techniques to use for reduction ('PCA', 't-SNE', 'UMAP', 'LDA').
        n_components: int = 2,                # Number of dimensions to reduce to.
        figsize: tuple = (12, 9),             # Size of the figure for each subplot.
        save_path: Optional[str] = None,      # Optional path to save the plot image.
        many_classes: bool = False,           # Flag to use enhanced plotting for many classes.
        grid_view: bool = False,              # Flag to plot all techniques in a single grid view.
        **kwargs: Any                         # Additional keyword arguments for dimensionality reduction methods.
    ) -> None:
    """
    Plots the combined latent space of real and synthetic data using specified dimensionality reduction techniques.
    """
    # Concatenate real and synthetic data
    combined_data = np.concatenate([real_data, synthetic_data], axis=0)

    # Create labels for real and synthetic data
    real_labels = np.zeros(real_data.shape[0], dtype=int)
    synthetic_labels = np.ones(synthetic_data.shape[0], dtype=int)
    combined_labels = np.concatenate([real_labels, synthetic_labels], axis=0)

    # Generate latent representations for the combined dataset
    latent_representations = encoder.predict(combined_data)

    # Handle case where encoder returns a tuple or list
    if isinstance(latent_representations, (list, tuple)):
        latent_representations = latent_representations[0]  # Assuming the first element is the required representation

    # Convert latent_representations to a NumPy array if it is a list
    latent_representations = np.array(latent_representations)

    # Reshape the latent representations to 2D: (samples, features)
    latent_representations = latent_representations.reshape(latent_representations.shape[0], -1)

    # Plot the latent space using the previously defined function
    reduce_dimensions_plot_latent_space(
        latent_representations=latent_representations,
        labels=combined_labels,
        techniques=techniques,
        n_components=n_components,
        figsize=figsize,
        save_path=save_path,
        many_classes=many_classes,
        grid_view=grid_view,
        class_names=["Real", "Synthetic"],
        **kwargs
    )

In [ ]:
# | export
def plot_combined_latent_space_with_labels(
        real_data: np.ndarray,                # Real data samples.
        synthetic_data: np.ndarray,           # Synthetic data samples generated by a model.
        real_labels: np.ndarray,              # Labels for the real data samples.
        encoder,                              # Encoder function or model that predicts latent space representations.
        techniques: List[str] = ['PCA'],      # Techniques to use for reduction ('PCA', 't-SNE', 'UMAP', 'LDA').
        n_components: int = 2,                # Number of dimensions to reduce to.
        figsize: tuple = (12, 9),             # Size of the figure for each subplot.
        real_colors: Optional[List[str]] = None,   # Optional list of colors for the real data labels. If None, use random colors.
        synthetic_color: str = 'red',         # Color for the synthetic data points.
        save_path: Optional[str] = None,      # Optional path to save the plot image.
        **kwargs: Any                         # Additional keyword arguments for dimensionality reduction methods.
    ) -> None:
    """
    Plots the combined latent space of real and synthetic data using specified dimensionality reduction techniques.
    The real data points are colored according to their labels, and the synthetic data points are overlaid in a new color.
    """
    # Concatenate real and synthetic data
    combined_data = np.concatenate([real_data, synthetic_data], axis=0)

    # Create labels for real and synthetic data
    synthetic_labels = np.full(synthetic_data.shape[0], -1, dtype=int)  # Use -1 to distinguish synthetic data
    combined_labels = np.concatenate([real_labels, synthetic_labels], axis=0)

    # Generate latent representations for the combined dataset
    latent_outputs = encoder.predict(combined_data)
    latent_representations = latent_outputs[0]  # Assuming the mean of the latent space is the first output

    models = {
        'PCA': PCA(n_components=n_components),
        't-SNE': TSNE(n_components=n_components, **kwargs),
        'UMAP': umap.UMAP(n_components=n_components, **kwargs),
        'LDA': LinearDiscriminantAnalysis(n_components=n_components)
    }

    for technique in techniques:
        model = models.get(technique)
        if not model:
            continue  # Skip if model not found in dictionary

        if technique == 'LDA':
            results = model.fit_transform(latent_representations, combined_labels)
        else:
            results = model.fit_transform(latent_representations)

        fig = plt.figure(figsize=figsize)
        if n_components == 1:
            ax = fig.add_subplot(111)
            real_data_points = ax.scatter(results[:real_data.shape[0]], np.zeros(real_data.shape[0]), c=real_labels, cmap='viridis' if real_colors is None else real_colors, s=30, label='Real Data')
            synthetic_data_points = ax.scatter(results[real_data.shape[0]:], np.zeros(synthetic_data.shape[0]), c=synthetic_color, s=30, label='Synthetic Data')
            ax.set_xlabel(f'{technique} Component 1')
        elif n_components == 2:
            ax = fig.add_subplot(111)
            real_data_points = ax.scatter(results[:real_data.shape[0], 0], results[:real_data.shape[0], 1], c=real_labels, cmap='viridis' if real_colors is None else real_colors, s=30, label='Real Data')
            synthetic_data_points = ax.scatter(results[real_data.shape[0]:, 0], results[real_data.shape[0]:, 1], c=synthetic_color, s=30, label='Synthetic Data')
            ax.set_xlabel(f'{technique} Dimension 1')
            ax.set_ylabel(f'{technique} Dimension 2')
        elif n_components == 3:
            ax = fig.add_subplot(111, projection='3d')
            real_data_points = ax.scatter(results[:real_data.shape[0], 0], results[:real_data.shape[0], 1], results[:real_data.shape[0], 2], c=real_labels, cmap='viridis' if real_colors is None else real_colors, s=30, label='Real Data')
            synthetic_data_points = ax.scatter(results[real_data.shape[0]:, 0], results[real_data.shape[0]:, 1], results[real_data.shape[0]:, 2], c=synthetic_color, s=30, label='Synthetic Data')
            ax.set_xlabel(f'{technique} Dimension 1')
            ax.set_ylabel(f'{technique} Dimension 2')
            ax.set_zlabel(f'{technique} Dimension 3')

        ax.set_title(f'Visualization with {technique}')
        plt.legend()
        if save_path:
            individual_save_path = f"{save_path}_{technique}.png"
            plt.savefig(individual_save_path)
            print(f"Saved plot to {individual_save_path}")
        plt.show()

## Exploring

In [39]:
#| export
def linear_interpolation(z1, z2, steps):
    """Perform linear interpolation between two points."""
    return np.linspace(z1, z2, steps)

def slerp(z1, z2, steps):
    """Perform spherical linear interpolation between two points."""
    z1_norm = z1 / np.linalg.norm(z1)
    z2_norm = z2 / np.linalg.norm(z2)
    dot_product = np.clip(np.dot(z1_norm, z2_norm), -1.0, 1.0)
    omega = np.arccos(dot_product)
    if omega == 0:
        return np.tile(z1, (steps, 1))
    sin_omega = np.sin(omega)
    return np.array([
        (np.sin((1 - t) * omega) / sin_omega) * z1 +
        (np.sin(t * omega) / sin_omega) * z2
        for t in np.linspace(0, 1, steps)
    ])

def interpolate_sample(centroids, granularity=10, variance=0.0):
    """
    Perform interpolating sampling between all pairs of centroids.

    Parameters:
    - centroids (np.ndarray): Array of shape (n_centroids, latent_dim).
    - granularity (int): Number of interpolation steps between each pair.
    - variance (float): Standard deviation for Gaussian sampling.

    Returns:
    - samples (np.ndarray): Array of sampled points.
    """
    latent_dim = centroids.shape[1]
    samples = []

    for z1, z2 in combinations(centroids, 2):
        if latent_dim <= 2:
            interpolated = linear_interpolation(z1, z2, granularity)
        else:
            interpolated = slerp(z1, z2, granularity)
        
        if variance > 0:
            noise = np.random.normal(0, variance, interpolated.shape)
            interpolated += noise
        
        samples.append(interpolated)
    
    if samples:
        return np.vstack(samples)
    else:
        return np.empty((0, latent_dim))

In [40]:
# Define example centroids for a 2-dimensional latent space
centroids = np.array([
    [1.0, 2.0],
    [3.0, 4.0],
    [5.0, 6.0]
])

granularity = 3
variance = 0.0  # Set to 0 for deterministic interpolation

sampled_points = interpolate_sample(centroids, granularity, variance)

# Define the expected sampled points manually for granularity=3
expected_data = np.array([
    [1.0, 2.0],
    [2.0, 3.0],
    [3.0, 4.0],
    [1.0, 2.0],
    [3.0, 4.0],
    [5.0, 6.0],
    [3.0, 4.0],
    [4.0, 5.0],
    [5.0, 6.0]
])

# Check the sampled points against the expected data
test_eq(sampled_points, expected_data)

In [41]:
#| export
def compute_centroids(latents, labels, method='mean', return_labels=False, **kwargs):
    """
    Compute the centroid of each class in the latent space using various methods.
    
    Parameters:
    - latents (np.ndarray): Array of shape (n_samples, latent_dim).
    - labels (np.ndarray): Array of shape (n_samples,) with class labels.
    - method (str): Method to compute centroids. Options: 'mean', 'median', 'geom_median', 'medoid', 'trimmed_mean', 'gmm'.
    - return_labels (bool): If True, also return the unique labels corresponding to the centroids.
    - kwargs: Additional arguments for specific methods.
    
    Returns:
    - centroids (np.ndarray): Array of shape (n_classes, latent_dim) containing centroids.
    - unique_labels (np.ndarray, optional): Array of shape (n_classes,) with unique class labels.
    """
    unique_labels = np.unique(labels)
    centroids = []
    
    for label in unique_labels:
        class_points = latents[labels == label]
        
        if method == 'mean':
            centroid = class_points.mean(axis=0)
        
        elif method == 'median':
            centroid = np.median(class_points, axis=0)
        
        elif method == 'geom_median':
            centroid = geometric_median(class_points, tol=kwargs.get('tol', 1e-5))
        
        elif method == 'medoid':
            centroid = compute_medoid(class_points)
        
        elif method == 'trimmed_mean':
            trim_ratio = kwargs.get('trim_ratio', 0.1)
            centroid = trimmed_mean_centroid(class_points, trim_ratio=trim_ratio)
        
        elif method == 'gmm':
            n_components = kwargs.get('n_components', 1)
            if n_components != 1:
                raise ValueError("GMM-based centroids require n_components=1 for simple centroid computation.")
            gmm = GaussianMixture(n_components=1)
            gmm.fit(class_points)
            centroid = gmm.means_[0]
        
        else:
            raise ValueError(f"Unsupported centroid computation method: {method}")
        
        centroids.append(centroid)
    
    centroids = np.array(centroids)
    
    if return_labels:
        return centroids, unique_labels
    else:
        return centroids

# Auxiliary Functions
def geometric_median(points, tol=1e-5):
    y = np.mean(points, axis=0)
    while True:
        D = distance.cdist([y], points, 'euclidean')[0]
        nonzeros = (D != 0)
        
        if not np.any(nonzeros):
            return y
        
        D = D[nonzeros]
        points_nonzero = points[nonzeros]
        y1 = np.sum(points_nonzero / D[:, np.newaxis], axis=0) / np.sum(1 / D)
        
        if np.linalg.norm(y - y1) < tol:
            return y1
        y = y1

def compute_medoid(points):
    dist_matrix = distance.cdist(points, points, 'euclidean')
    medoid_index = np.argmin(dist_matrix.sum(axis=1))
    return points[medoid_index]

def trimmed_mean_centroid(points, trim_ratio=0.1):
    trimmed_points = []
    for dim in range(points.shape[1]):
        sorted_dim = np.sort(points[:, dim])
        trim = int(trim_ratio * len(sorted_dim))
        trimmed_dim = sorted_dim[trim: -trim]
        trimmed_points.append(trimmed_dim)
    return np.array([np.mean(dim) for dim in trimmed_points])

In [42]:
#| hide

# Define example latent representations
latents = np.array([
    [1.0, 2.0],
    [1.5, -1.0],
    [2.0, -1.0],
    [1.2, 2.2],
    [1.4, 1.8],
    [1.7, 7.0]
])

# Define corresponding labels
labels = np.array([1,1,1,2,2,3])

# Compute centroids without returning labels
centroids = compute_centroids(latents, labels)

# Define the expected centroids
expected_centroids = np.array([
    [1.5, 0.],        # Centroid for label 1
    [1.3, 2.],        # Centroid for label 2
    [1.7, 7.]         # Centroid for label 2
])

# Check the computed centroids against the expected data
test_close(centroids, expected_centroids)

In [43]:
#| hide
import nbdev; nbdev.nbdev_export()